In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import LogFormatterSciNotation
import matplotlib.colors as colors

import basal_melt_param.map_plot_functions as mapfunc

from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as fa
import mpl_toolkits.axisartist.grid_finder as gf

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
nemo_run0 = 'OPM021'
if nemo_run0 in ['OPM031-1','OPM031-2']:
    nemo_run = 'OPM031'
else:
    nemo_run = nemo_run0

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
plot_path = '/bettik/burgardc/PLOTS/NN_plots/2D_patterns/'
inputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'

In [ ]:
melt_rate_2D_NEMO = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_timmean.nc')
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
#mod_size_list = ['medium_onlyTSisfdraft','medium_onlyTSdraftandslope','extra_large_onlyTSdraftandslope','medium_TSdraftbotandiceddandwcd','medium_TSdraftslopereldGL','extra_large_TSdraftslopereldGL']#['mini', 'small', 'medium', 'large'] #, 'extra_large'
mod_size_list = ['xsmall96_newbasic2','small_newbasic2','large_newbasic2'] #,#['mini', 'small', 'medium', 'large'] #, 'extra_large'

norm_method = 'std'
TS_opt = 'extrap' #'whole' #'extrap'

melt_2D_param_CVtime_list = []
melt_2D_param_CVisf_list = []
    
for mod_size in mod_size_list:
    print(mod_size)

    outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'

    new_path_output = outputpath_melt_nn+'CV_TBLOCKS/'
    melt_rate_2D_nn = xr.open_dataset(new_path_output+'evalmetrics_2D_CV_'+mod_size+'_'+TS_opt+'_norm'+norm_method+'_'+nemo_run+'.nc')
    melt_rate_2D_nn_timmean = melt_rate_2D_nn['predicted_melt'].mean('time').reindex_like(file_isf['ISF_mask'])
    melt_2D_param_CVtime_list.append(melt_rate_2D_nn_timmean.assign_coords({'crossval': 'time'}))
    
    new_path_output = outputpath_melt_nn+'CV_ISF/'
    melt_rate_2D_nn = xr.open_dataset(new_path_output+'evalmetrics_2D_CV_'+mod_size+'_'+TS_opt+'_norm'+norm_method+'_'+nemo_run+'.nc')
    melt_rate_2D_nn_timmean = melt_rate_2D_nn['predicted_melt'].mean('time').reindex_like(file_isf['ISF_mask'])
    melt_2D_param_CVisf_list.append(melt_rate_2D_nn_timmean.assign_coords({'crossval': 'isf'}))

melt_2D_CVtime_all = xr.concat(melt_2D_param_CVtime_list, dim='nn_model')
melt_2D_CVisf_all = xr.concat(melt_2D_param_CVisf_list, dim='nn_model')
melt_2D_all = xr.concat([melt_2D_CVtime_all, melt_2D_CVisf_all], dim='crossval')
melt_2D_all = melt_2D_all.assign_coords(nn_model=mod_size_list)

In [ ]:
ref_melt_2D = melt_rate_2D_nn['reference_melt'].where(file_isf_mask).mean('time')

In [ ]:
grounded_msk03 = file_isf['ground_mask'].where(file_isf['ground_mask']==0,3)
grounded_msk = (grounded_msk03.where(grounded_msk03!=3,1)-1)*-1

In [ ]:
icesheet_msk_0inf = file_isf_mask.where(file_isf_mask!=1,0)
icesheet_msk = icesheet_msk_0inf.where(icesheet_msk_0inf < 1, 1)

In [ ]:
box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))
box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')
box1_msk = box1.where(box1==1,0)

TRY TAYLOR DIAGRAM FOR ONE PARAM AND ONE ICE SHELF

In [ ]:
def taylor_diagram(refstd, stddev, corrcoef, fig=None, levels=5, extend=False):
    
    ccolors = ['blue','green','magenta']
    srange = [0,2] # in units of refstd

    tr = PolarAxes.PolarTransform()
    
    # Correlation labels
    rlocs = np.array([0, 0.2, 0.4, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1])
    if extend:
        # Diagram extended to negative correlations
        ccoef_max = np.pi
        rlocs = NP.concatenate((-rlocs[:0:-1], rlocs))
    else:
        # Diagram limited to positive correlations
        ccoef_max = np.pi/2
    tlocs = np.arccos(rlocs)        # Conversion to polar angles
    gl1 = gf.FixedLocator(tlocs)    # Positions
    tf1 = gf.DictFormatter(dict(zip(tlocs, map(str, rlocs))))

    # Standard deviation axis extent (in units of reference stddev)
    stdmin = srange[0] * refstd
    stdmax = srange[1] * refstd

    ghelper = fa.GridHelperCurveLinear(
        tr,
        extremes=(0, ccoef_max, stdmin, stdmax),
        grid_locator1=gl1, tick_formatter1=tf1)
    
    if fig == None:
        fig = plt.figure()
    
    ax = fa.FloatingSubplot(fig, 122, grid_helper=ghelper)
    fig.add_subplot(ax)

    # Adjust axes
    ax.axis["top"].set_axis_direction("bottom")   # "Angle axis"
    ax.axis["top"].toggle(ticklabels=True, label=True)
    ax.axis["top"].major_ticklabels.set_axis_direction("top")
    ax.axis["top"].label.set_axis_direction("top")
    ax.axis["top"].label.set_text("Correlation")

    ax.axis["left"].set_axis_direction("bottom")  # "X axis"
    ax.axis["left"].label.set_text("Standard deviation")

    ax.axis["right"].set_axis_direction("top")    # "Y-axis"
    ax.axis["right"].toggle(ticklabels=True)
    ax.axis["right"].major_ticklabels.set_axis_direction(
        "bottom" if extend else "left")

    if stdmin:
        ax.axis["bottom"].toggle(ticklabels=False, label=False)
    else:
        ax.axis["bottom"].set_visible(False)          # Unused

    ax0 = ax.get_aux_axes(tr)   # Polar coordinates

    # Add reference point and stddev contour
    l, = ax0.plot([0], refstd, 'k*',
                      ls='', ms=10, label='Reference')
    t = np.linspace(0, ccoef_max)
    r = np.zeros_like(t) + refstd
    ax0.plot(t, r, 'k--', label='_')

    # Collect sample points for latter use (e.g. legend)
    samplePoints = [l]

    ### NOW FEED THE DATA
    
    for i,cc in enumerate(corrcoef):
        if i > 2:
            ccol = ccolors[i-3]
        else:
            ccol = ccolors[i]
        l, = ax0.plot(np.arccos(corrcoef[i]), stddev[i], color=ccol, marker='o')  # (theta, radius)
    samplePoints.append(l)

    ax.grid()
    
    rs, ts = np.meshgrid(np.linspace(stdmin, stdmax),
                         np.linspace(0, ccoef_max))
    # Compute centered RMS difference
    rms = np.sqrt(refstd**2 + rs**2 - 2*refstd*rs*np.cos(ts))

    contours = ax0.contour(ts, rs, rms, levels, linestyle='--')
        
    return fig

In [ ]:
kisf = 31
if kisf == 11:
    kisf_mask = (file_isf_mask==11) | (file_isf_mask==21)
else:
    kisf_mask = file_isf_mask==kisf

ref_melt_isf = ref_melt_2D.where(kisf_mask,drop=True)
param_melt_isf = melt_2D_all.where(kisf_mask,drop=True)


In [ ]:
ref_std = ref_melt_isf.std().values.flatten()[0]
stddev = param_melt_isf.std(['x','y']).values.flatten() 
ccoeff = xr.corr(ref_melt_isf,param_melt_isf,dim=['x','y']).values.flatten()

In [ ]:
ref_std[0]

In [ ]:
stddev

In [ ]:
fig = taylor_diagram(ref_std, stddev, ccoeff ,fig=None,extend=False)

In [ ]:
fig_66 = taylor_diagram(ref_std, stddev, ccoeff ,fig=fig_66,extend=False)